In [1]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import jieba.posseg as pseg
import jieba
import re
import string
import csv
import keras as layers
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from gensim.models import word2vec
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import load_model
from keras.models import Sequential

Using TensorFlow backend.
D:\Anaconda3\envs\tens\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#全局变量，一些参数，这样后面比较好改动
#这几个参数都能够进行调整，以及没有使用到的参数都能用来调整模型
# 设置词语向量的维度,这个参数暂时还不适合新闻正文的大小
num_features = 200 #原来是100
# 保证被拷问词语的最低频度
min_word_count = 5
# 设置并行化训练使用CPU计算核心数量
num_workers = 4
# 设置词语上下文窗口的大小
context = 5
#句子最大长度兼向量的维度
maxLen=50

In [3]:
#去除停用词的两个函数
def stopwordslist(stopfname): #读入停用词
    stopwords=[line.strip() for line in open(stopfname,encoding='UTF-8').readlines()]
    return stopwords

def deleteStop(sentence,stopfname):   #根据读入的停用词，把分词后的数据中的停用词删掉
    stopwords=stopwordslist(stopfname)
    outstr=""
    for i in sentence:
        if i not in stopwords and i!="\n":
            outstr+=i
    return outstr

In [4]:
#进行分词用的函数
def wordCut(Text,stopfname):#这里传的是前面的listText
    Mat=[]
    for rec in Text:
        senten=[]
        rec=re.sub('[%s]' % re.escape(string.punctuation),'',str(rec))  #去掉逗号，句号等标点符号
        fenci=jieba.lcut(rec)    #精准模式分词
        stc=deleteStop(fenci,stopfname)    #去停用词
        seg_list=pseg.cut(stc)   #标注词性，紧接着去掉对分类来说没用的词，比如人名地名等
        for word,flag in seg_list:
            if flag not in ["nr","ns","nt","nz","m","f","ul","l","r","t"]:
                senten.append(word)
        Mat.append(senten)
    return Mat

In [5]:
#读取数据，并且划分出来训练集和测试集

#读取数据
data_path="./data.txt"
text=[]
with open(data_path,"r",encoding="utf-8") as f:
    for line in f:
        line.strip()   #去掉换行符
        text.append(line)
text.pop(0)     #把label和text那行字符去掉，直接处理

#划分测试集和训练集，data本身一共7766条数据，其中5322条积极的，2444条消极的，决定训练集：测试集=8：2进行分配
#故共选择4256条积极，1952条消极作为训练集(6208条)，其余作为测试集（1558条）
train_data=[]
test_data=[]
#生成训练集
for i in range(0,4256):
    train_data.append(text[i])
for i in range(5322,7274):
    train_data.append(text[i])
#生成测试集
for i in range(4256,5322):
    test_data.append(text[i])
for i in range(7274,7766):
    test_data.append(text[i])
    
random.shuffle(train_data)  #打乱数据，以防止之后训练模型的时候效果不好
random.shuffle(test_data)

In [6]:
#使用分词函数，并去除停用词

#先将标签和文本分开
train_text=[]
test_text=[]
train_label=[]
test_label=[]
#训练集
for line in train_data:
    temp=line.split(',')
    train_label.append(temp[0])
    train_text.append(temp[1].strip()) #之前的换行符没去干净，这一步又去掉了一下
#测试集
for line in test_data:
    temp=line.split(',')
    test_label.append(temp[0])
    test_text.append(temp[1].strip()) #之前的换行符没去干净，这一步又去掉了一下

In [7]:
#对文本进行分词，训练集和测试集都需要
train_cut=wordCut(train_text,"./stopword.txt") #数据类型是二维list，一行是一个list，每个词组是其中一个元素
test_cut=wordCut(test_text,"./stopword.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\AAAAA\AppData\Local\Temp\jieba.cache
Loading model cost 0.670 seconds.
Prefix dict has been built successfully.


In [8]:
#训练集
train_ccut=[]
test_ccut=[]
for line in train_cut:
    st=""
    for j in line:
        st+=j
        st+=" "
    train_ccut.append(st)
# print(type(train_ccut))   #数据类型是list，每一个元素是一句话的分词结果，并且用空格隔开

#测试集
for line in test_cut:
    st=""
    for j in line:
        st+=j
        st+=" "
    test_ccut.append(st)

In [9]:
#数据预处理
#标签，使用LabelEncoder对数据标签进行规格化处理
def encodeLabel(data):
    #规格化处理
    le=LabelEncoder()
    resultLabel=le.fit_transform(data)#这里把label变成了数字的形式
    return resultLabel

In [10]:
#fit_on_texts函数可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小
#给每个词按照词频进行编号，返回编号后的“词典”
def getNum(word):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(word)
    vocab = tokenizer.word_index  # 得到每个词的编号，这里的vocab已剔除掉stoplist了
    return vocab

In [11]:
#word2vec的训练,将训练好的模型写入filename中，后面可以直接拿出来进行使用
#参数中的words是用来训练模型的分好词后的数据
def w2v(filename,words):
    # word2vec模型建立
    model=word2vec.Word2Vec(words,sg=1,size=num_features,window=5,min_count=5,negative=3,sample=0.001,hs=1,workers=4)
    # 强制单位归一化
    model.init_sims(replace=True)
    # 输入一个路径，保存模型
    model.save(filename)
    # model.wv.save_word2vec_format('这里写一个文件路径和名字',binary=False)  #这里是存下来模型自己可以看看的，好像是这样

In [12]:
#正文内容的编码
#特征数字编号，不足在前面补0
def getCode_Text(ww):
    tokenizer = Tokenizer()
    tID = tokenizer.texts_to_sequences(ww)
    tSeq = pad_sequences(tID, maxlen=maxLen)
    return tSeq

#标签的热独码,返回编好码的数据
def getCode_Label(label):
    tCate = to_categorical(label)  # 将标签转换为ont-hot编码,有10种，所以参数传的10
    return tCate

In [13]:
from numpy import nan as NaN
#一些数据处理相关的东西
word=[]
for i in train_ccut:
    word.append(i)
for j in test_ccut:
    word.append(j)

#这个地方是去除word中空的数据，不然之后使用Tokenizer时会因为存在空数据而报错
count=0
for i in word:
#     print(i)
    if i is "":
        word.pop(count)
    count+=1

In [14]:
# 把正文内容处理为向量，直接使用Tokenizer提供的函数进行实现
tokenizer = Tokenizer()
tokenizer.fit_on_texts(word)
vocab = tokenizer.word_index
trainID = tokenizer.texts_to_sequences(train_ccut)
traindata = pad_sequences(trainID, maxlen=maxLen)
testID = tokenizer.texts_to_sequences(test_ccut)
testdata = pad_sequences(testID, maxlen=maxLen)
# 把标签处理为数字，使用现成的库中的函数进行实现的
trainL=encodeLabel(train_label)
testL=encodeLabel(test_label)
trl=getCode_Label(trainL)
tel=getCode_Label(testL)

In [15]:
#这下面就是模型的东西了
#首先是word2vec模型
model=word2vec.Word2Vec(word,sg=1,size=num_features,window=5,min_count=5,negative=3,sample=0.001,hs=1,workers=4)
model.init_sims(replace=True)
    # 输入一个路径，保存模型
model.save('./w2vmodel')

In [16]:
#这下面就是模型的东西了
#其次是cnn相关的
from keras.models import Model
w2v_model = word2vec.Word2Vec.load('./w2vmodel') #取出之前保存的word2vec模型
embedding_matrix = np.zeros((len(vocab) + 1, num_features))  # 前面的vacab是前面编号好的那些，后面的参数是表示维度是100维
for word, i in vocab.items():
    try:  # 防止有没有找到的词时出错
        embedding_vector = w2v_model[str(word)]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue

# 模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接
main_input = Input(shape=(maxLen,), dtype='float64')
# 词嵌入（使用预训练的词向量）
embedder = Embedding(len(vocab) + 1, num_features, input_length=maxLen, weights=[embedding_matrix], trainable=False)
#embedder = Embedding(len(vocab) + 1, 300, input_length=50, trainable=False)
embed = embedder(main_input)
# 词窗大小分别为1,2,3
cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
cnn1 = MaxPooling1D(pool_size=38)(cnn1)
cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
cnn2 = MaxPooling1D(pool_size=37)(cnn2)
cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
cnn3 = MaxPooling1D(pool_size=36)(cnn3)

#新加几个卷积层和池化层
# cnn4 = Conv1D(256, 6, padding='same', strides=1, activation='relu')(embed)
# cnn4 = MaxPooling1D(pool_size=34)(cnn4)
# cnn5 = Conv1D(256, 7, padding='same', strides=1, activation='relu')(embed)
# cnn5 = MaxPooling1D(pool_size=35)(cnn5)
# cnn6 = Conv1D(256, 8, padding='same', strides=1, activation='relu')(embed)
# cnn6 = MaxPooling1D(pool_size=36)(cnn6)


# 合并三个模型的输出向量
cnn = concatenate([cnn1,cnn2,cnn3], axis=-1)
flat = Flatten()(cnn)
drop = Dropout(0.2)(flat)  #这里是dropout层，目的是防止过拟合
main_output = Dense(units=2, activation='softmax')(drop)
model = Model(inputs=main_input, outputs=main_output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
history=model.fit(traindata, trl, batch_size=800, epochs=20,validation_split=0.2) #训练模型，分批进行训练，一共训练20次，每次800条数据，预留20%的数据用于对当前的模型进行测试
model.save("./textCNNmodel") #保存模型

Train on 4966 samples, validate on 1242 samples
Epoch 1/20
4966/4966 [==============================] - 13s - loss: 0.6666 - acc: 0.6422 - val_loss: 0.6341 - val_acc: 0.7061
Epoch 2/20
4966/4966 [==============================] - 12s - loss: 0.6105 - acc: 0.7358 - val_loss: 0.5980 - val_acc: 0.7134
Epoch 3/20
4966/4966 [==============================] - 12s - loss: 0.5722 - acc: 0.7447 - val_loss: 0.5803 - val_acc: 0.7174
Epoch 4/20
4966/4966 [==============================] - 12s - loss: 0.5459 - acc: 0.7479 - val_loss: 0.5703 - val_acc: 0.7230
Epoch 5/20
4966/4966 [==============================] - 12s - loss: 0.5300 - acc: 0.7573 - val_loss: 0.5701 - val_acc: 0.7246
Epoch 6/20
4966/4966 [==============================] - 12s - loss: 0.5203 - acc: 0.7642 - val_loss: 0.5668 - val_acc: 0.7311
Epoch 7/20
4966/4966 [==============================] - 12s - loss: 0.5109 - acc: 0.7763 - val_loss: 0.5669 - val_acc: 0.7303
Epoch 8/20
4966/4966 [==============================] - 13s - loss: 0.

In [18]:
result = model.predict(testdata)  # 预测样本属于每个类别的概率
result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签
print(result_labels)
y_predict = list(map(str, result_labels))#把预测结果转变为list型的数据

[1 1 1 ..., 1 1 1]


In [19]:
#计算准确率
from sklearn.metrics import accuracy_score
print('准确率', accuracy_score(test_label, y_predict))

准确率 0.713735558408
